<a href="https://colab.research.google.com/github/k-l-s-h/Eye-Prediction-Model/blob/main/eye_disease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install dlib


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import cv2
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Flatten
from tensorflow.keras.layers import Conv2D,MaxPool2D
from tensorflow.keras.layers import Input, Dense
from keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

from tensorflow.keras.applications.vgg16 import VGG16
import pandas as pd
import numpy as np
import os
from numpy import argmax

In [ ]:
import zipfile
with zipfile.ZipFile("archive.zip","r") as zip_ref:
    zip_ref.extractall("targetdir")

In [ ]:
PATH = "./targetdir/Eye_diseases"
data_dir_list = os.listdir(PATH)
data_dir_list

['Crossed_Eyes',
 'Bulging_Eyes',
 'Cataracts',
 'Eye_diseases',
 'Glaucoma',
 'Uveitis']

In [ ]:
img_rows=224
img_cols=224
num_channel=3

num_epoch = 50
batch_size = 32

img_data_list=[]
classes_names_list=[]
target_column=[]
for dataset in data_dir_list:
    classes_names_list.append(dataset)
    print("Getting image from {} folder".format(dataset))
    img_list= os.listdir(PATH +"/"+ dataset)
    for img in img_list:
        if img.split('.')[-1] == 'jpeg':
            input_img = cv2.imread(PATH +"/"+dataset+"/"+img)
            input_img_resize=cv2.resize(input_img,(img_rows,img_cols))
            img_data_list.append(input_img_resize)
            target_column.append(dataset)

Getting image from Crossed_Eyes folder
Getting image from Bulging_Eyes folder
Getting image from Cataracts folder
Getting image from Eye_diseases folder
Getting image from Glaucoma folder
Getting image from Uveitis folder


In [ ]:
num_classes = len(classes_names_list)
print("num_classes",num_classes)
img_data = np.array(img_data_list) # convert images in numpy array
img_data = img_data.astype('float32')
img_data /= 255
print("Shape of image data",img_data.shape)
num_of_samples = img_data.shape[0]
input_shape = img_data[0].shape
print("number of samples",num_of_samples)
print("target column before encoding",target_column)

num_classes 6
Shape of image data (385, 224, 224, 3)
number of samples 385
target column before encoding ['Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes'

In [ ]:
Labelencoder = LabelEncoder()
target_column = Labelencoder.fit_transform(target_column)
np.unique(target_column)

array([0, 1, 2, 3, 4])

In [ ]:
target_column

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

In [ ]:
target_column_hotcoded = to_categorical(target_column,num_classes)
X,Y = shuffle(img_data,target_column_hotcoded,random_state=2)
X_train,X_temp,y_train,y_temp = train_test_split(X,Y,test_size=0.3,random_state=2)
X_test,X_val,y_test,y_val = train_test_split(X_temp,y_temp,test_size=0.3,random_state=2)

In [ ]:
first_Mod = Sequential()

first_Mod.add(Conv2D(64,(3,3),activation='relu',input_shape=input_shape))
first_Mod.add(Conv2D(64,(3,3),activation='relu'))
first_Mod.add(MaxPool2D(pool_size=(2,2)))
first_Mod.add(Dropout(0.2))

first_Mod.add(Conv2D(128,(3,3),activation='relu'))
first_Mod.add(Conv2D(128,(3,3),activation='relu'))
first_Mod.add(MaxPool2D(pool_size=(2,2)))
first_Mod.add(Dropout(0.2))

first_Mod.add(Flatten())
first_Mod.add(Dense(128,activation='relu'))
first_Mod.add(Dropout(0.2))
first_Mod.add(Dense(num_classes,activation='softmax'))
first_Mod.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
first_Mod.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 220, 220, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 110, 110, 64)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 110, 110, 64)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 108, 108, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 106, 106, 128)     147584    
                                                        

In [ ]:
from keras.callbacks import ModelCheckpoint
filepath = "eye-model-{epoch:02d}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=False, mode='max')
callbacks_list = [checkpoint]
hist = first_Mod.fit(X_train,y_train,batch_size=batch_size,epochs=num_epoch,verbose=1,callbacks=callbacks_list,validation_data=(X_test,y_test))
score = first_Mod.evaluate(X_test,y_test,batch_size=batch_size)
print('Test Loss',score[0])
print("Test Accuracy",score[1])

Epoch 1/50
9/9 [==============================] - ETA: 0s - loss: 3.0859 - accuracy: 0.3123 
Epoch 1: saving model to eye-model-01.hdf5
9/9 [==============================] - 177s 19s/step - loss: 3.0859 - accuracy: 0.3123 - val_loss: 1.5776 - val_accuracy: 0.2099
Epoch 2/50
9/9 [==============================] - ETA: 0s - loss: 1.4886 - accuracy: 0.3941 
Epoch 2: saving model to eye-model-02.hdf5
9/9 [==============================] - 170s 19s/step - loss: 1.4886 - accuracy: 0.3941 - val_loss: 1.5287 - val_accuracy: 0.3580
Epoch 3/50
9/9 [==============================] - ETA: 0s - loss: 1.4625 - accuracy: 0.4424 
Epoch 3: saving model to eye-model-03.hdf5
9/9 [==============================] - 172s 19s/step - loss: 1.4625 - accuracy: 0.4424 - val_loss: 1.5364 - val_accuracy: 0.3704
Epoch 4/50
9/9 [==============================] - ETA: 0s - loss: 1.3983 - accuracy: 0.4572 
Epoch 4: saving model to eye-model-04.hdf5
9/9 [==============================] - 170s 19s/step - loss: 1.3983 -